# Data Collector for Earnings Event-Driven RL Trading

This notebook processes raw price data and earnings dates into a structured dataset ready for environment building.

## Overview
1. Load raw price and earnings data
2. Build event windows (N days before and after earnings)
3. Calculate technical features
4. Generate training/test splits
5. Export to CSV format for environment building

## Step 1: Import Libraries and Configuration

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
from pathlib import Path

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Step 2: Configuration Parameters

In [12]:
# ============ Data Paths ============
RAW_PRICE_DATA = "MAG7_hourly_5yr.csv"  # Change this to your file path
EARNINGS_DATA = "mag7_earnings_dates_2019_2024.csv"  # Change this to your file path
OUTPUT_CSV = "earnings_events_data.csv"  # Output file

# ============ Event Window Configuration ============
DAYS_BEFORE = 5      # Days before earnings
DAYS_AFTER = 5       # Days after earnings
TOTAL_WINDOW = DAYS_BEFORE + DAYS_AFTER + 1  # Total days = 11

# ============ Feature Calculation ============
MOMENTUM_WINDOW = 3   # Calculate momentum using past 3 days
VOLATILITY_WINDOW = 3  # Calculate volatility using past 3 days

# ============ Trading Costs ============
TRANSACTION_COST = 0.0005  # 0.05% (round-trip)

# ============ Train/Test Split ============
TRAIN_TEST_SPLIT = 0.80  # 80% training, 20% test

print("Configuration:")
print(f"  Price Data: {RAW_PRICE_DATA}")
print(f"  Earnings Data: {EARNINGS_DATA}")
print(f"  Output CSV: {OUTPUT_CSV}")
print(f"  Event Window: {DAYS_BEFORE} days before + {DAYS_AFTER} days after")
print(f"  Train/Test Split: {TRAIN_TEST_SPLIT:.0%}/{1-TRAIN_TEST_SPLIT:.0%}")

Configuration:
  Price Data: MAG7_hourly_5yr.csv
  Earnings Data: mag7_earnings_dates_2019_2024.csv
  Output CSV: earnings_events_data.csv
  Event Window: 5 days before + 5 days after
  Train/Test Split: 80%/20%


## Step 3: Load Raw Data

In [13]:
logger.info("Loading raw data...")

# Load price data
price_data = pd.read_csv(RAW_PRICE_DATA)
price_data['timestamp'] = pd.to_datetime(price_data['timestamp'])
price_data = price_data.sort_values(['ticker', 'timestamp']).reset_index(drop=True)

# Load earnings dates
earnings_dates = pd.read_csv(EARNINGS_DATA)
earnings_dates['earnings_date'] = pd.to_datetime(earnings_dates['earnings_date'])

logger.info(f"✓ Loaded {len(price_data):,} price records")
logger.info(f"✓ Loaded {len(earnings_dates)} earnings release dates")

print("\nPrice Data Sample:")
print(price_data.head())
print(f"\nPrice Data Info:")
print(f"  - Date Range: {price_data['timestamp'].min()} to {price_data['timestamp'].max()}")
print(f"  - Tickers: {price_data['ticker'].unique().tolist()}")
print(f"  - Total Records: {len(price_data):,}")

print("\nEarnings Data Sample:")
print(earnings_dates.head())
print(f"\nEarnings Data Info:")
print(f"  - Date Range: {earnings_dates['earnings_date'].min()} to {earnings_dates['earnings_date'].max()}")
print(f"  - Total Dates: {len(earnings_dates)}")

2025-12-05 01:43:06,376 - INFO - Loading raw data...
2025-12-05 01:43:06,548 - INFO - ✓ Loaded 160,574 price records
2025-12-05 01:43:06,548 - INFO - ✓ Loaded 144 earnings release dates



Price Data Sample:
  symbol                 timestamp    open    high     low   close   volume  \
0   AAPL 2019-01-01 00:00:00+00:00  157.92  158.18  157.91  158.18  14683.0   
1   AAPL 2019-01-02 09:00:00+00:00  154.40  154.70  153.01  154.70  24033.0   
2   AAPL 2019-01-02 10:00:00+00:00  154.50  154.70  154.35  154.46   8605.0   
3   AAPL 2019-01-02 11:00:00+00:00  154.58  154.65  154.10  154.46  18522.0   
4   AAPL 2019-01-02 12:00:00+00:00  154.50  155.35  154.50  154.50  85291.0   

   trade_count        vwap ticker  
0         85.0  157.989075   AAPL  
1        180.0  154.224457   AAPL  
2         67.0  154.546683   AAPL  
3        162.0  154.360502   AAPL  
4        862.0  154.835064   AAPL  

Price Data Info:
  - Date Range: 2019-01-01 00:00:00+00:00 to 2024-12-31 00:00:00+00:00
  - Tickers: ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']
  - Total Records: 160,574

Earnings Data Sample:
  ticker earnings_date
0   AAPL    2024-04-30
1   AAPL    2024-08-01
2   AAPL   

## Step 4: Build Event Windows

In [14]:
logger.info("Building event windows...")

all_events_data = []
valid_events_count = 0
skipped_events_count = 0

for _, earnings_row in earnings_dates.iterrows():
    ticker = earnings_row['ticker']
    earnings_date = pd.Timestamp(earnings_row['earnings_date'])
    
    # Get price data for this company
    ticker_data = price_data[price_data['ticker'] == ticker].copy()
    
    # Define event window
    window_start = earnings_date - timedelta(days=DAYS_BEFORE)
    window_end = earnings_date + timedelta(days=DAYS_AFTER)
    
    # Extract window data - convert to date for comparison (avoid timezone issues)
    ticker_data['timestamp_date'] = ticker_data['timestamp'].dt.date
    earnings_start_date = window_start.date()
    earnings_end_date = window_end.date()
    
    window_data = ticker_data[
        (ticker_data['timestamp_date'] >= earnings_start_date) & 
        (ticker_data['timestamp_date'] <= earnings_end_date)
    ].copy()
    
    # Check data completeness
    if len(window_data) < 50:  # Approximately 5 days * 10 hours
        skipped_events_count += 1
        continue
    
    # Sort by timestamp
    window_data = window_data.sort_values('timestamp').reset_index(drop=True)
    window_data = window_data.drop('timestamp_date', axis=1)  # Remove helper column
    
    # Add event metadata
    window_data['ticker_event'] = ticker
    window_data['earnings_date'] = earnings_date
    window_data['window_start'] = window_start
    window_data['window_end'] = window_end
    window_data['event_id'] = valid_events_count
    
    all_events_data.append(window_data)
    valid_events_count += 1

logger.info(f"✓ Built {valid_events_count} valid event windows")
logger.info(f"⊘ Skipped {skipped_events_count} events due to insufficient data")

print(f"\nEvent Windows Summary:")
print(f"  - Valid Events: {valid_events_count}")
print(f"  - Skipped Events: {skipped_events_count}")
print(f"  - Total Data Points in Events: {sum(len(e) for e in all_events_data):,}")

2025-12-05 01:43:06,558 - INFO - Building event windows...
2025-12-05 01:43:07,822 - INFO - ✓ Built 138 valid event windows
2025-12-05 01:43:07,822 - INFO - ⊘ Skipped 6 events due to insufficient data



Event Windows Summary:
  - Valid Events: 138
  - Skipped Events: 6
  - Total Data Points in Events: 14,815


## Step 5: Calculate Technical Features

In [15]:
def calculate_features(window_data, earnings_date):
    """
    Calculate technical features for an event window.
    
    Returns dict with:
    - momentum: Pre-earnings momentum
    - volatility: Pre-earnings volatility
    - avg_volume: Average volume
    - pre_close: Pre-earnings close price
    - window_return: Window return
    """
    try:
        # Convert earnings_date to date for comparison
        if hasattr(earnings_date, 'date'):
            earnings_date_compare = earnings_date.date()
        else:
            earnings_date_compare = earnings_date
        
        # Separate pre-earnings data
        if 'timestamp_date' in window_data.columns:
            pre_earnings = window_data[window_data['timestamp_date'] < earnings_date_compare]
        else:
            window_data_copy = window_data.copy()
            window_data_copy['temp_date'] = window_data_copy['timestamp'].dt.date
            pre_earnings = window_data_copy[window_data_copy['temp_date'] < earnings_date_compare]
            pre_earnings = pre_earnings.drop('temp_date', axis=1)
        
        if len(pre_earnings) < 20:
            return None
        
        # 1. Pre-earnings momentum (return over past N days)
        pre_returns = pre_earnings['close'].pct_change().dropna()
        momentum = pre_returns.tail(MOMENTUM_WINDOW).sum()
        
        # 2. Volatility (std dev over past N days)
        volatility = pre_returns.tail(VOLATILITY_WINDOW).std()
        
        # 3. Average volume
        avg_volume = pre_earnings['volume'].tail(VOLATILITY_WINDOW).mean()
        
        # 4. Pre-earnings closing price
        pre_close = pre_earnings['close'].iloc[-1]
        
        # 5. Window return (full window)
        pre_price = pre_earnings['close'].iloc[0]
        
        if 'timestamp_date' in window_data.columns:
            post_earnings = window_data[window_data['timestamp_date'] > earnings_date_compare]
        else:
            window_data_copy = window_data.copy()
            window_data_copy['temp_date'] = window_data_copy['timestamp'].dt.date
            post_earnings = window_data_copy[window_data_copy['temp_date'] > earnings_date_compare]
            post_earnings = post_earnings.drop('temp_date', axis=1)
        
        if len(post_earnings) > 0:
            post_price = post_earnings['close'].iloc[-1]
            window_return = (post_price - pre_price) / pre_price
        else:
            window_return = 0
        
        return {
            'momentum': momentum,
            'volatility': volatility if volatility > 0 else 0.01,
            'avg_volume': avg_volume,
            'pre_close': pre_close,
            'window_return': window_return
        }
    
    except Exception as e:
        logger.warning(f"Feature calculation failed: {e}")
        return None

logger.info("Calculating features for all events...")

# Add features to each event
processed_events = []
for event_data in all_events_data:
    ticker = event_data['ticker_event'].iloc[0]
    earnings_date = event_data['earnings_date'].iloc[0]
    
    features = calculate_features(event_data, earnings_date)
    
    if features is not None:
        # Add features to each row in the event
        event_data['momentum'] = features['momentum']
        event_data['volatility'] = features['volatility']
        event_data['avg_volume_event'] = features['avg_volume']
        event_data['pre_close'] = features['pre_close']
        event_data['window_return'] = features['window_return']
        
        processed_events.append(event_data)

logger.info(f"✓ Calculated features for {len(processed_events)} events")

print(f"\nFeature Calculation Summary:")
print(f"  - Events with Features: {len(processed_events)}")
print(f"  - Total Data Points: {sum(len(e) for e in processed_events):,}")

2025-12-05 01:43:07,830 - INFO - Calculating features for all events...
2025-12-05 01:43:08,053 - INFO - ✓ Calculated features for 138 events



Feature Calculation Summary:
  - Events with Features: 138
  - Total Data Points: 14,815


## Step 6: Create Train/Test Split

In [16]:
# Combine all events into single dataframe
combined_data = pd.concat(processed_events, ignore_index=True)

# Get unique events for train/test split
unique_events = combined_data[['event_id', 'earnings_date']].drop_duplicates('event_id')

# Split events (not individual rows)
train_size = int(len(unique_events) * TRAIN_TEST_SPLIT)
train_event_ids = unique_events.iloc[:train_size]['event_id'].values
test_event_ids = unique_events.iloc[train_size:]['event_id'].values

# Add split label
combined_data['split'] = combined_data['event_id'].apply(
    lambda x: 'train' if x in train_event_ids else 'test'
)

logger.info(f"✓ Train/Test Split:")
logger.info(f"  - Training events: {len(train_event_ids)}")
logger.info(f"  - Test events: {len(test_event_ids)}")

print(f"\nTrain/Test Split Summary:")
print(f"  - Training Events: {len(train_event_ids)}")
print(f"  - Test Events: {len(test_event_ids)}")
print(f"  - Training Rows: {len(combined_data[combined_data['split'] == 'train']):,}")
print(f"  - Test Rows: {len(combined_data[combined_data['split'] == 'test']):,}")

print(f"\nData Sample:")
print(combined_data.head(10))

2025-12-05 01:43:08,089 - INFO - ✓ Train/Test Split:
2025-12-05 01:43:08,089 - INFO -   - Training events: 110
2025-12-05 01:43:08,090 - INFO -   - Test events: 28



Train/Test Split Summary:
  - Training Events: 110
  - Test Events: 28
  - Training Rows: 11,788
  - Test Rows: 3,027

Data Sample:
  symbol                 timestamp      open      high       low     close  \
0   AAPL 2024-04-25 08:00:00+00:00  168.8300  169.1400  168.6200  168.8500   
1   AAPL 2024-04-25 09:00:00+00:00  168.8200  169.1500  168.8200  169.1300   
2   AAPL 2024-04-25 10:00:00+00:00  169.0000  169.0900  168.8100  168.8100   
3   AAPL 2024-04-25 11:00:00+00:00  168.7400  169.3100  168.6300  169.1500   
4   AAPL 2024-04-25 12:00:00+00:00  168.7966  169.9600  168.1400  168.3400   
5   AAPL 2024-04-25 13:00:00+00:00  168.3500  170.6100  167.8900  169.0100   
6   AAPL 2024-04-25 14:00:00+00:00  169.0200  169.4900  168.4700  168.5750   
7   AAPL 2024-04-25 15:00:00+00:00  168.5700  168.8700  168.1511  168.5500   
8   AAPL 2024-04-25 16:00:00+00:00  168.5400  169.5383  168.5000  169.3599   
9   AAPL 2024-04-25 17:00:00+00:00  169.3600  169.6100  168.8600  169.5501   

      vo

## Step 7: Feature Statistics

In [17]:
# Get unique features per event
event_features = combined_data.groupby('event_id').agg({
    'momentum': 'first',
    'volatility': 'first',
    'avg_volume_event': 'first',
    'window_return': 'first',
    'split': 'first'
}).reset_index()

print("\n=== Feature Statistics ===")
print("\nTraining Set Features:")
train_features = event_features[event_features['split'] == 'train']
print(train_features[['momentum', 'volatility', 'window_return']].describe())

print("\nTest Set Features:")
test_features = event_features[event_features['split'] == 'test']
print(test_features[['momentum', 'volatility', 'window_return']].describe())

print(f"\nWindow Return Distribution:")
print(f"  - Positive Returns: {(event_features['window_return'] > 0).sum()} events")
print(f"  - Negative Returns: {(event_features['window_return'] < 0).sum()} events")
print(f"  - Mean Return: {event_features['window_return'].mean():.4f}")
print(f"  - Median Return: {event_features['window_return'].median():.4f}")


=== Feature Statistics ===

Training Set Features:
         momentum  volatility  window_return
count  110.000000  110.000000     110.000000
mean     0.001282    0.002467       0.016606
std      0.007561    0.003193       0.086952
min     -0.038086    0.000095      -0.185032
25%     -0.000925    0.000817      -0.037597
50%      0.001308    0.001486       0.017194
75%      0.003705    0.002634       0.054115
max      0.036428    0.019010       0.342879

Test Set Features:
        momentum  volatility  window_return
count  28.000000   28.000000      28.000000
mean    0.005437    0.004203       0.008454
std     0.009150    0.003863       0.114102
min    -0.013961    0.000343      -0.263640
25%    -0.000304    0.001458      -0.042332
50%     0.003732    0.002493      -0.001243
75%     0.009840    0.006458       0.078706
max     0.029653    0.013927       0.268753

Window Return Distribution:
  - Positive Returns: 76 events
  - Negative Returns: 62 events
  - Mean Return: 0.0150
  - Median

## Step 8: Prepare CSV for Environment Building

In [18]:
# Select relevant columns for environment
csv_columns = [
    'ticker_event',
    'earnings_date',
    'window_start',
    'window_end',
    'event_id',
    'timestamp',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'momentum',
    'volatility',
    'avg_volume_event',
    'pre_close',
    'window_return',
    'split'
]

output_data = combined_data[csv_columns].copy()

# Sort by event and timestamp
output_data = output_data.sort_values(['event_id', 'timestamp']).reset_index(drop=True)

# Save to CSV
output_data.to_csv(OUTPUT_CSV, index=False)

logger.info(f"✓ Saved CSV to {OUTPUT_CSV}")

print(f"\n CSV Export Complete!")
print(f"\nFile: {OUTPUT_CSV}")
print(f"\nColumns:")
for i, col in enumerate(csv_columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nDataset Size:")
print(f"  - Total Rows: {len(output_data):,}")
print(f"  - Total Columns: {len(csv_columns)}")
print(f"  - File Size: {Path(OUTPUT_CSV).stat().st_size / 1024:.2f} KB")

print(f"\nFirst Few Rows:")
print(output_data.head())

print(f"\nLast Few Rows:")
print(output_data.tail())

2025-12-05 01:43:08,268 - INFO - ✓ Saved CSV to earnings_events_data.csv



 CSV Export Complete!

File: earnings_events_data.csv

Columns:
   1. ticker_event
   2. earnings_date
   3. window_start
   4. window_end
   5. event_id
   6. timestamp
   7. open
   8. high
   9. low
  10. close
  11. volume
  12. momentum
  13. volatility
  14. avg_volume_event
  15. pre_close
  16. window_return
  17. split

Dataset Size:
  - Total Rows: 14,815
  - Total Columns: 17
  - File Size: 2847.57 KB

First Few Rows:
  ticker_event earnings_date window_start window_end  event_id  \
0         AAPL    2024-04-30   2024-04-25 2024-05-05         0   
1         AAPL    2024-04-30   2024-04-25 2024-05-05         0   
2         AAPL    2024-04-30   2024-04-25 2024-05-05         0   
3         AAPL    2024-04-30   2024-04-25 2024-05-05         0   
4         AAPL    2024-04-30   2024-04-25 2024-05-05         0   

                  timestamp      open    high     low   close    volume  \
0 2024-04-25 08:00:00+00:00  168.8300  169.14  168.62  168.85   38844.0   
1 2024-04-25 09:00:

## Step 9: Data Quality Check

In [19]:
print("\n=== Data Quality Check ===")

# Check for missing values
missing_values = output_data.isnull().sum()
print(f"\nMissing Values:")
for col in missing_values[missing_values > 0].index:
    print(f"  - {col}: {missing_values[col]}")
if missing_values.sum() == 0:
    print("  ✓ No missing values")

# Check for duplicates
duplicates = output_data.duplicated().sum()
print(f"\nDuplicate Rows: {duplicates}")

# Check split distribution
print(f"\nSplit Distribution:")
print(output_data['split'].value_counts())

# Check data types
print(f"\nData Types:")
print(output_data.dtypes)

# Check for outliers
print(f"\nPrice Range Check:")
print(f"  - Close Price Min: ${output_data['close'].min():.2f}")
print(f"  - Close Price Max: ${output_data['close'].max():.2f}")
print(f"  - Volume Min: {output_data['volume'].min():,}")
print(f"  - Volume Max: {output_data['volume'].max():,}")

print(f"\n Data Quality Check Complete!")


=== Data Quality Check ===

Missing Values:
  ✓ No missing values

Duplicate Rows: 0

Split Distribution:
split
train    11788
test      3027
Name: count, dtype: int64

Data Types:
ticker_event                     object
earnings_date            datetime64[ns]
window_start             datetime64[ns]
window_end               datetime64[ns]
event_id                          int64
timestamp           datetime64[ns, UTC]
open                            float64
high                            float64
low                             float64
close                           float64
volume                          float64
momentum                        float64
volatility                      float64
avg_volume_event                float64
pre_close                       float64
window_return                   float64
split                            object
dtype: object

Price Range Check:
  - Close Price Min: $92.04
  - Close Price Max: $3702.00
  - Volume Min: 100.0
  - Volume Max: 58,777,3

## Summary Statistics

In [20]:
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

print(f"\n Dataset Statistics:")
print(f"  - Total Events: {output_data['event_id'].nunique()}")
print(f"  - Training Events: {len(output_data[output_data['split'] == 'train']['event_id'].unique())}")
print(f"  - Test Events: {len(output_data[output_data['split'] == 'test']['event_id'].unique())}")
print(f"  - Total Data Points: {len(output_data):,}")
print(f"  - Unique Tickers: {output_data['ticker_event'].nunique()}")
print(f"  - Date Range: {output_data['timestamp'].min()} to {output_data['timestamp'].max()}")

print(f"\n Output File:")
print(f"  - Filename: {OUTPUT_CSV}")
print(f"  - Size: {Path(OUTPUT_CSV).stat().st_size / (1024*1024):.2f} MB")
print(f"  - Format: CSV (can be opened in Excel, pandas, etc.)")

print(f"\n Data collection complete! Ready for environment building.")
print("="*80)


FINAL SUMMARY

 Dataset Statistics:
  - Total Events: 138
  - Training Events: 110
  - Test Events: 28
  - Total Data Points: 14,815
  - Unique Tickers: 6
  - Date Range: 2019-01-25 00:00:00+00:00 to 2024-11-05 23:00:00+00:00

 Output File:
  - Filename: earnings_events_data.csv
  - Size: 2.78 MB
  - Format: CSV (can be opened in Excel, pandas, etc.)

 Data collection complete! Ready for environment building.
